### Machine Learning-based Data Analysis with Case Study
#### copyright @ Minseo Park
### [Practice 4] : Recommendation
    1. 문제정의하기(Problem Define)
    2. 라이브러리 불러오기(Libraries Setting)
    3. 데이터 수집하기(Data Collection)
    4. 데이터 탐색하기(Data Exploration)
    5. 전처리하기(Preprocessing)
    6. 모델링하기(Modeling)
        - 6.1 품목 추천(Item based Recommendations)
        - 6.2 사용자에게 개인맞춤 추천하기 (Personalized Recommendations)

#### [1]: Problem Define : 문제정의
- 애니메이션 추천 시스템 - 협업 필터링 Matrix Factorization : 애니메이션을 추천합니다.

- Data: MyAnimeList Dataset
     - y = f(x)
     - y: 애니메이션 추천
     - x: 애니메이션 데이터, 사용자 데이터

#### [2] 라이브러리 불러오기(Libraries Setting)

In [ ]:
import numpy as np # Numeric Python 
import pandas as pd # Data Processing and Database
import matplotlib.pyplot as plt # Visualization
import seaborn as sns # Visualization

from sklearn.decomposition import TruncatedSVD # Recommendations
from scipy.sparse.linalg import svds #Recommendations

[SVD :Singular Value Decomposition(특이값 분해)]

- 왜 사용할까요? 
- 특이값 분해의 목적: 특이값 분해는 행렬을 분해하는 방법 중 하나입니다. 그렇다면 멀쩡한 행렬을 대체 왜 분해하는 것일까요? 
    - 이해를 돕기 위해 다른 유명한 분해를 예로 들어보겠습니다. 
    - 우리가 아는 가장 유명한 분해는 중학교때 배운 **인수분해**입니다.
    - **인수분해의 목적**은 무엇일까요? 
        - 인수분해는 이차방정식의 해를 구하기 위한 도구로 쓰입니다.
        - **이차방정식 자체만 보고 해를 바로 구하는건 꽤 어렵다고 생각합니다.**
        - 따라서 수학자들은 **인수분해**라는 방법을 통해 이차방정식의 해를 좀 더 쉽게 구할 수 있었습니다.
    - 그렇다면 **특이값 분해**는 어떨까요? 
        - 인수분해가 이차방정식의 해를 구하기 위한 도구로 쓰이는 것처럼 **특이값 분해도 행렬을 차원축소 하기위한 도구**로 쓰입니다. 
    - **개념상 비유한다면, SVD == 인수분해** 입니다.
        - 실제로 주성분분석(PCA, Principle Component Analysis)과 같은 차원축소 분야에서는 특이값 분해가 흔하게 쓰입니다.
    
- 특이값 분해의 활용: 특이값 분해는 차원축소를 하는데 사용됩니다. 
    - 데이터를 나타내는 행렬 A가 존재한다고 합시다. 차원축소는 말그대로 차원을 축소시키는 것입니다.  
        - 데이터 전체 공간의 차원보다 낮은 차원으로 근사시켜 적합(fit)시킬 수 있는 차원이 낮은 공간을 찾는 것입니다. 
        - 만약 A가 d차원이라고 했을때, A를 k차원(d>k)으로 축소시킨 행렬을 B라고 한다면, 결국 특이값 분해는 차원축소 행렬 B를 찾는 데 사용되는 것입니다. 
        - 이 때, 우리는 k를 정할 수 있습니다. 말그대로 1차원으로 줄일 수도 있고, 2차원으로 줄일 수도 있습니다. 

- 참고) Truncated SVD는 Sigma 행렬에 있는 대각원소 ,즉 특이값 중 상위 일부 데이터만 추출해 분해하는 방식을 의미합니다.

#### [3] 데이터 수집하기(Data Collection)

In [ ]:
rating_data = pd.read_csv('./data/rating_complete.csv') ## 사용자가 애니메이션에 대해 매긴 점수
anime_data = pd.read_csv('./data/anime_with_synopsis.csv') ## 애니메이션에 대한 정보 
rating_data.head()

해석) 
- user_id가 같다는 의미는 한 사람임을 알려줍니다. 
    - 한 사람이 여러 애니메이션을 볼 수 있습니다. 
    - 한 사람이 여러 애니메이션들에 대해 점수를 매길 수 있습니다.

In [ ]:
anime_data.head()

해석) 
- 두 개의 파일은 사용자-평점 데이터와 애니메이션 데이터로 나뉘어져 있습니다.   
- 이 두개의 파일은 공통적으로 anime_id를 가지고 있습니다.
    - anime_id를 활용하면 하나로 합칠 수 있습니다. 
- 데이터 분석하려면, 통합 테이블을 만들어야 합니다.  
    - 분석할 것들을 잘 추려서 하나의 테이블/데이터베이스로 만드는 것이 중요합니다. 
    - 이 때 anime_id를 활용하면 좋습니다.

#### [4] 데이터 탐색하기(Data Exploration)

- 데이터의 각 변수(features, attributes, columns, x들)의 기본 정보 및 특성을 자세히 살펴봅니다.

### [데이터 셋 구조]

In [ ]:
print(anime_data.shape)
print(rating_data.shape)

### [데이터 타입]

In [ ]:
anime_data.info()

### [데이터 타입]

In [ ]:
rating_data.info()

### [데이터 통계]

In [ ]:
anime_data.describe()

### [데이터 통계]

In [ ]:
rating_data.describe()

In [ ]:
rating_data.head(10)

In [ ]:
userId_duplicate= rating_data.drop_duplicates(['user_id'])
userId_duplicate.head()

해석) 
- 한사람이 매겼던 애니메이션 점수 중 가장 위의 값, 1개만 가져옵니다.

In [ ]:
userId_duplicate.shape

해석) 
- 사용자 수를 확인합니다.

#### [5] 전처리하기(Preprocessing)

In [ ]:
# 평점이 -1인 데이터 제거 (평가하지 않은 경우)
rating_data = rating_data[rating_data['rating'] != -1]
rating_data.head()

### inplace=True: 명령어를 실행한 후 메소드가 적용된 데이터 프레임으로 반환한다.
### 즉, 삭제 메소드를 실행했다면 반환값은 컬럼이 삭제된 Dataframe이 반환한다.
### axis = 1: 열, axis = 0 : 행을 따라 동작합니다.

In [ ]:
# 필요한 컬럼만 선택
anime_data_clean = anime_data[['MAL_ID', 'Name', 'Score', 'Genres', 'Episodes', 'Type']].copy()
anime_data_clean = anime_data_clean.rename(columns={'MAL_ID': 'anime_id'})
anime_data_clean.head()

In [ ]:
# 데이터 샘플링 (메모리 효율성)
# 평점이 많은 인기 애니메이션만 사용
anime_rating_counts = rating_data['anime_id'].value_counts()
popular_animes = anime_rating_counts[anime_rating_counts >= 100].index

# 평가를 많이 한 활성 사용자만 사용
user_rating_counts = rating_data['user_id'].value_counts()
active_users = user_rating_counts[user_rating_counts >= 50].index

# 필터링
rating_data_sampled = rating_data[
    (rating_data['anime_id'].isin(popular_animes)) &
    (rating_data['user_id'].isin(active_users))
]

print(f"Sampled Data Shape: {rating_data_sampled.shape}")
print(f"Users: {rating_data_sampled['user_id'].nunique()}")
print(f"Animes: {rating_data_sampled['anime_id'].nunique()}")

In [ ]:
user_anime_rating = rating_data_sampled.pivot_table('rating', index = 'user_id', 
                     columns='anime_id').fillna(0)

해석) 
- rating 를 value 로 사용합니다. 
- user_id 를 인덱스로 사용합니다. 
- 애니메이션 ID(anime_id)에 대한 rating(점수)를 넣어줍니다. 
- rating이 없는 경우는 value를 0으로 할당합니다

In [ ]:
user_anime_rating.shape
### 사용자 수, 애니메이션 갯수

In [ ]:
user_anime_rating.head()

- **사용자-애니메이션 데이터**의 pivot table을 만들었습니다.  
- 다음에는, 이제 사용자-애니메이션 기준의 데이터를 **애니메이션-사용자** 기준으로 만들어서 **특정 '애니메이션'과 비슷한 애니메이션을 추천**해주는 로직을 구현해봅니다.
- 기준을 바꾸어 줍니다. 
    - 현재 사용자 기준에서 **애니메이션을 기준으로 전치행렬**을 만들어 줍니다.

## 행이 애니메이션, 열이 사용자

In [ ]:
anime_user_rating = user_anime_rating.values.T
anime_user_rating.shape

#### [6] 모델링하기(Modeling)

#### [6.1]  품목 추천(Item based Recommendations)

In [ ]:
SVD = TruncatedSVD(n_components=12)
matrix = SVD.fit_transform(anime_user_rating) 
matrix.shape

### anime_user_rating = user_anime_rating.values.T
### 가로: 애니메이션 세로: 사용자

**truncated SVD**는 
#####  - 시그마 행렬의 대각원소(특이값) 가운데 상위 n개만 골라낸 것입니다. 
#####  - 기존 행렬 A의 성질을 100% 원복할 수는 없지만, (그 만큼 데이터 정보를 압축) 행렬 A와 거의 근사한 값이 나오게 됩니다.

In [ ]:
matrix[0]

해석) 
- 12개의 component로 차원을 축소했습니다.  
- 사용자 점수를 12개의 점수로 압축하였습니다.

In [ ]:
corr = np.corrcoef(matrix)
corr.shape

[알아두기]피어슨 상관계수 
- np.corrcoef(): 피어슨 상관계수 값을 계산해 줍니다. 
- 행을 기준으로 값들을 변수로 생각해서, 그 변수에 대한 상관계수를 구하게 됩니다. (애니메이션을 기준으로)
- 애니메이션-애니메이션을 비교하여, 애니메이션들 간의 상관관계를 봅니다.

[쉬어가기]상관관계 (correlation) 
- 두 변수간에 어떤 선형적 관계를 갖고 있는지 분석하는 방법입니다. 
- 두 변수는 서로 독립적인 관계로 부터, 서로 상관된 관계일수 있으면, 이때 두 변수간의 **관계의 강도**를  **상관관계(Correlation, Correlation Coefficient)** 라 합니다. 
- 상관관계의 정도를 파악하는 **상관계수(Correlation Coefficient)는 두 변수간의 연관된 정도를 나타낼 뿐 인과관계를 설명하는 것은 아닙니다.**

In [ ]:
anime_title = user_anime_rating.columns
anime_title_list = list(anime_title)
sample_anime = anime_title_list[0]
sample_index = 0
sample_index

해석)
- 샘플 애니메이션 인덱스를 찾습니다.

In [ ]:
anime_title_list[0:20]
### 처음 20개의 애니메이션 리스트를 출력합니다.

In [ ]:
corr_sample = corr[sample_index]
list(anime_title[(corr_sample >= 0.9)])[:10]

해석) 
- 샘플 애니메이션을 기준으로 비슷한 애니메이션을 뽑아봤습니다. 
    - 가장 비슷한 애니메이션 10개까지만 뽑습니다.  
    - 상관계수가 0.9 이상인 유사한 작품들을 찾습니다.

#### [6.2] 사용자에게 개인맞춤 추천하기 (Personalized Recommendations)
- 위에서는 하나의 애니메이션에 대해서 비슷한 애니메이션을 추천해주는 것을 적용했습니다.   
- 하지만, 보통 추천 시스템은 사용자에게 맞춤 추천을 해주어야 합니다.  
- 사용자에게 맞춤 추천을 해주기 위해서 협업 필터링 행렬 분해를 적용해보겠습니다.

[리뷰] Data Upload : 
- 데이터를 상기시키기 위해서 다시 한번 해 줍니다. 
- 헷갈리지 않기 위해서 data frame 이름을 다르게 선언합니다.

In [ ]:
df_ratings = rating_data_sampled.copy()
df_animes = anime_data_clean.copy()
df_ratings.head()

In [ ]:
df_animes.head()

- 사용자 기준 애니메이션에 대한 평점 테이블을 만듭니다.

In [ ]:
df_user_anime_ratings = df_ratings.pivot_table(
    values='rating',
    index='user_id',
    columns='anime_id'
).fillna(0)
df_user_anime_ratings.shape

In [ ]:
df_user_anime_ratings.iloc[:10, 0:5]
### 사용자 10명과 애니메이션 5개에 대한 점수를 보여줍니다.

In [ ]:
df_user_anime_ratings.head()

해석) 

- 여기까지는 앞서 했던 것과 똑같습니다. 
- 사용자-애니메이션 pivot table을 만들었습니다.

- 이제 아래와 같이 데이터를 조금 변경해서 진행하겠습니다.
    1. pivot table을 **matrix**로 변환합니다.
    2. np.mean(axis = 1)을 통해 사용자들이 매기는 **애니메이션들에 대한 평점 평균**을 구함 : **user_ratings_mean**
        - axis인자를 통해서 이것의 방향성을 지정합니다. 
        - axis = 1: 열
    3. 1에서 구한 값과 2에서 구한 값을 빼서 **(특정 사용자가 매긴 점수- 사용자가 애니메이션들에 대해서 매긴 점수들의 평균 값)** 으로 데이터 값을 변경합니다.
        - 평균을 기점으로 **+** 이면 긍정적인 점수, 
        - **-** 이면 부정적인 점수를 가지는 것을 의미합니다.

In [ ]:
### matrix는 pivot_table 값을 numpy matrix로 만듭니다. 
matrix = df_user_anime_ratings.to_numpy()

### user_ratings_mean은 한 사용자가 모든 애니메이션들에 대해 매긴 점수의 평균을 의미합니다.
### axis = 1: 열, axis = 0 행을 따라 동작합니다. 
user_ratings_mean = np.mean(matrix, axis = 1) 

### matrix_user_mean : 
### ** 사용자가 준 점수 - 사용자가 애니메이션들에 준 점수들의 평균 ** 을 구합니다.
### 평균과 얼마나 차이나는지가 들어가게 됩니다.
matrix_user_mean = matrix - user_ratings_mean.reshape(-1, 1)

[알아두기] reshape(-1,1(정수)) -> 열이 1로 나열하세요!!
- reshape() 의 행(row) 차원 위치에 '-1'을 넣으면 어떻게 재구조화가 되는지 살펴보겠습니다. 
    - 총 12개의 원소가 들어있는 배열 x에 대해서 x.reshape(-1, 정수) 를 해주면 '열(column)' 차원의 '정수'에 따라서 12개의 원소가 빠짐없이 배치됩니다.
    - '-1'이 들어가 있는 '행(row)' 의 개수가 가변적으로 정해집니다.

In [ ]:
matrix.shape

In [ ]:
user_ratings_mean.shape

해석) 
- user_ratings_mean은 각 사용자가 모든 애니메이션에 준 점수의 평균을 의미합니다.
- 따라서 평균이므로 각 사용자들마다 1개의 데이터 밖에 없습니다.

In [ ]:
user_ratings_mean
### 사용자별 애니메이션 평균 평점을 의미합니다.

In [ ]:
matrix_user_mean.shape

##### matrix_user_mean
- 사용자가 준 점수- 사용자가 모든 애니메이션들에 매긴 점수의 평균 을 구합니다.
- 사용자 * 애니메이션 매트릭스 입니다. 
- 평균과 얼마나 차이나는지가 들어가게 됩니다.

In [ ]:
pd.DataFrame(matrix_user_mean, columns = df_user_anime_ratings.columns).head()

해석 1) 
- 여기까지 진행하면 초기에 만들었던 user-anime pivot table 값이 matrix_user_mean 변경되었습니다. 즉, 아래와 같이 변경된 것이죠.

    1. 사용자들이 애니메이션에 대해 평점을 매긴 값이 존재합니다.
    2. 사용자들의 각각 애니메이션에 대한 평균 평점을 합니다.
    3. 사용자들의 애니메이션에 대한 호감도 점수를 구합니다.
        - 얼마나 더 좋아하는지를 빈도로 표현할 수 있습니다. 
        - 해당 프로세스는 정규화의 효과를 만듭니다.

In [ ]:
### scipy에서 제공해주는 SVD
### SVD(Singular Value Decomposition), 특이값 분해:  m x n 크기의 데이터 행렬을 차수를 줄여 간소화 하는 방법 중 한개 입니다.
# U 행렬, sigma 행렬, V 전치 행렬을 반환.

U, sigma, Vt = svds(matrix_user_mean, k = 12)

해석 2) 
**SVD를 이용해 Matrix Factorization**을 진행해봅니다. 
- 앞서서는 scikit learn을 이용해 TruncatedSVD를 이용했는데요. 이번에는 scipy를 이용해 Truncated SVD를 구해봅니다.
- 차이점은 
    - scikit learn에서 제공해주는 TruncatedSVD는 U, Sigma, Vt 반환 값을 제공하지 않습니다.   
    - 하지만, Scipy를 이용하면 이 반환값들을 제공받을 수 있습니다.
    - Scipy에서 제공해주는 SVD는 scipy.sparse.linalg.svds를 이용하면 됩니다.

In [ ]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

현재 이 Sigma 행렬은 0이 아닌 값만 1차원 행렬로 표현된 상태입니다.  
**즉, 0이 포함된 대칭행렬로 변환할 때는 numpy의 diag를 이용해야 합니다.**

In [ ]:
sigma = np.diag(sigma)
sigma.shape

##### 해석하기) 현재 까지 상황을 정리하면 아래와 같습니다.   

1. 원본 user-anime 평점 행렬입니다.
2. 이를 user의 평균 점수를 빼서 matrix_user_mean 이라는 행렬로 만듭니다.
3. 2번의 값을 SVD를 적용해 U, Sigma, Vt 행렬을 구합니다.
4. Sigma 행렬로 차원 축소합니다.

##### [쉬어가기]  
- (행렬 검증하기) 자! 이제 여기서 matrix_user_mean을 SVD를 적용해 분해를 한 상태입니다. 이제, 다시 원본 행렬로 복구시켜야겠죠?
- 원본 행렬로 복구시키는 방법은 아래와 같습니다.
- U, Sigma, Vt의 내적을 수행합니다. 
    - np.dot(np.dot(U, sigma), Vt)를 수행하면 됩니다. 
    - 그리고 아까 사용자 평균을 빼주었으니 여기서는 더해줍니다.

In [ ]:
# U, Sigma, Vt의 내적을 수행하면, 다시 원본 행렬로 복원이 됩니다. 
# 내적 값에 + 사용자 평균 rating을 적용합니다. 
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, 
                            columns = df_user_anime_ratings.columns,
                            index = df_user_anime_ratings.index)
df_svd_preds.head()

In [ ]:
df_svd_preds.shape

#### 추천하는 함수 만들기 
- 인자: 사용자-애니메이션 점수(예측값을 갖는) 테이블, 사용자 아이디(user_id), 원본 애니메이션 정보 테이블(ori_animes_df), 원본 평점 테이블(ori_ratings_df), 애니메이션 추천 개수(num_recommendations=5) 등을 받습니다.
- 사용자 아이디에 SVD로 나온 결과의 애니메이션 평점이 가장 높은 데이터 순으로 정렬합니다. 
- 사용자가 본 데이터를 제외합니다.
- 사용자가 안 본 애니메이션에서 평점이 높은 것을 추천합니다.

In [ ]:
def recommend_animes(df_svd_preds, user_id, ori_animes_df, 
                     ori_ratings_df, num_recommendations):
    
    ### 사용자가 예측 데이터프레임에 있는지 확인
    if user_id not in df_svd_preds.index:
        print(f"사용자 ID {user_id}가 존재하지 않습니다.")
        return None, None
    
    ### 최종적으로 만든(전처리된) sorted_user_predictions에서 
    ### 사용자에 따라 애니메이션 데이터 정렬합니다. 
    sorted_user_predictions = df_svd_preds.loc[user_id].sort_values(ascending=False)
    
    ### 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아냅니다. 
    user_data = ori_ratings_df[ori_ratings_df['user_id'] == user_id]
    
    ### 해당 사용자의 원본 평점 데이터와 원본 애니메이션 데이터를 합칩니다. 
    ### 애니메이션 평균이 높은 순으로 정렬(sorting)합니다.
    user_history = user_data.merge(ori_animes_df, on = 'anime_id').sort_values(['rating'], ascending=False)
    
    ### 애니메이션 데이터에서 사용자가 본 애니메이션 데이터를 제외한 데이터를 필터링합니다. 
    ### 안 본 애니메이션을 추천해 주어야 합니다. 
    ### 따라서, 보지 않은 애니메이션들만 필터링 합니다.
    recommendations = ori_animes_df[~ori_animes_df['anime_id'].isin(user_history['anime_id'])]
    
    ### 안 본 애니메이션과 사용자의 애니메이션 평점이 높은 순으로 정렬된 데이터와 
    ### recommendations을 합칩니다.
    ### 즉, 평점이 높으면서 아직 보지 않은 애니메이션을 추출합니다. 
    recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(),
                             on = 'anime_id')
    
    ### 컬럼 이름 바꾸고 정렬합니다. 
    ### 이전에 봤던 애니메이션과, 추천하는 애니메이션을 함께 보여줍니다.
    recommendations = recommendations.rename(columns = {user_id: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      
    return user_history, recommendations

In [ ]:
# 샘플 사용자 선택
sample_user_id = df_user_anime_ratings.index[0]

already_rated, predictions = recommend_animes(df_svd_preds, sample_user_id, df_animes, df_ratings, 10)

In [ ]:
already_rated.head(10)

In [ ]:
predictions
### 사용자에게 추천하는 애니메이션입니다.

#### 결론)
- 사용자 별로 다르게 추천됨을 알수 있습니다. 
- 사용자 맞춤 추천을 할 수 있습니다.

#### 모델 저장

In [ ]:
import pickle

model_data = {
    'U': U,
    'sigma': sigma,
    'Vt': Vt,
    'anime_ids': list(df_user_anime_ratings.columns),
    'user_ratings_mean': user_ratings_mean,
    'anime_info': df_animes.set_index('anime_id').to_dict('index')
}

with open('./data/svd_model.pkl', 'wb') as f:
    pickle.dump(model_data, f)

print("모델이 './data/svd_model.pkl'에 저장되었습니다.")